In [25]:
import pandas as pd
import numpy as np
import json
from scipy.sparse import hstack
import time
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.preprocessing.data import OneHotEncoder
from sklearn import preprocessing

# data读取和整理

In [26]:
#读取各表数据合并
goods_label_df = pd.read_excel('./good_feature_data/goods_label_1.xlsx')
goods_name_sku_match_df = pd.read_excel('./good_feature_data/goods_name_sku_match.xls')
goods_match_on_sale_df = pd.read_excel('./good_feature_data/goods_match_on_sale_s.xls')

In [27]:
#将goods_match_df进行在售筛选，但是发现只有多规格可以筛选，套餐未知原因不能筛选
goods_df = pd.merge(goods_match_on_sale_df,goods_name_sku_match_df,how= 'left',left_on='goods_slug',right_on='slug')
p_on_sale_df = pd.read_excel('/Users/wjj/Desktop/jupyter/Boohee_tag_recommend/personas/XGBoost/on_sale_goods/p_20191113.xls')
s_on_sale_df = pd.read_excel('/Users/wjj/Desktop/jupyter/Boohee_tag_recommend/personas/XGBoost/on_sale_goods/s_20191113.xls')
del p_on_sale_df['goods_name']
del s_on_sale_df['goods_name']
on_sale_df = pd.concat([s_on_sale_df,p_on_sale_df])
on_sale_list = on_sale_df['slug'].tolist()
s_slug = goods_match_on_sale_df[goods_match_on_sale_df['s_slug'].isin(on_sale_list)]
goods_match_on_sale_df['p_slug'] = goods_match_on_sale_df['p_slug'].astype(str)
p_slug = goods_match_on_sale_df[goods_match_on_sale_df['p_slug'].isin(on_sale_list)]
s_slug.to_excel('./s_slug.xls',sheet_name='Sheet1',index = False)
goods_match_on_sale_df = goods_match_on_sale_df[['s_slug','goods_slug']]
goods_match_on_sale_df['goods_slug'] = goods_match_on_sale_df['goods_slug'].astype(str)
goods_label_df['g_slug'] = goods_label_df['g_slug'].astype(str)


# 商品Embedding

In [28]:
#读取商品标签
goods_feature_df = pd.read_excel('./good_feature_data/s_feature_1.xlsx')
#有空格
goods_feature_df = goods_feature_df.replace('\u3000',0)

In [29]:
goods_feature_df

,goods_slug,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,s_ES_20190724,0,false,EA代餐,商品,自发货,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,1.0
1,s_qianka_1000k_qingliang_20190703,0,true,EF轻麦片,商品,自发货,0.0,0.0,1,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
2,s_qianka_1000k_qingliang_20190703,0,false,EF轻饮,商品,自发货,0.0,0.0,1,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
3,s_qianka_1000k_qingliang_20190703,0,false,EF轻肉食,商品,自发货,1.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
4,s_qianka_1000k_qingliang_20190703,0,false,EA小助手,商品,自发货,0.0,0.0,0,0.0,...,1,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
5,s_qianka_1000k_qingliang_20190703,0,false,EF轻烘焙,商品,自发货,0.0,1.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
6,s_qianka_1000k_qingliang_20190703,0,true,EF轻主食,商品,自发货,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
7,s_qianka_1000k_qingliang_20190703,0,false,EF轻肉食,商品,自发货,1.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
8,s_qianka_1000k_qingliang_20190703,0,false,EF轻小食,商品,自发货,0.0,0.0,0,1.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
9,s_qianka_1000k_qingliang_20190703,0,false,EF轻主食,商品,自发货,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0


In [30]:
goods_feature_df.loc[goods_feature_df[2]==' false',2]= 1
goods_feature_df.loc[goods_feature_df[2]==' true',2]= 0
goods_feature_df.loc[goods_feature_df[4]=='设备',4]= 1
goods_feature_df.loc[goods_feature_df[4]==' 商品',4]= 0
goods_feature_df.loc[goods_feature_df[5]==' 第三方发货',5]= 1
goods_feature_df.loc[goods_feature_df[5]==' 自发货',5]= 0
goods_feature_df = goods_feature_df.fillna(0)

In [31]:
for i in range(6,92):
    goods_feature_df[i] = goods_feature_df[i].astype(int)

In [32]:
#3特征的onehot
goods_feature_df = goods_feature_df.join(pd.get_dummies(goods_feature_df[3]))
del goods_feature_df[3]

In [33]:
#统一到多规格粒度
goods_feature_group_df = goods_feature_df.groupby('goods_slug').sum()

In [34]:
#所有非零值都认为1
goods_feature_group_df[goods_feature_group_df != 0] = 1

In [35]:
def price_process(price):
    price = int(price)
    if price==0:
        return 'a'
    elif price<20:
        return 'b'
    elif price<40:
        return 'c'
    elif price<60:
        return 'd'
    elif price<80:
        return 'e'
    elif price<100:
        return 'f'
    elif price<120:
        return 'g'
    elif price<140:
        return 'h'
    elif price<160:
        return 'i'
    elif price<180:
        return 'j'
    elif price<200:
        return 'k'
    elif price<220:
        return 'l'
    elif price<240:
        return 'm'
    elif price<260:
        return 'n'
    elif price<280:
        return 'o'
    elif price<300:
        return 'p'
    elif price<350:
        return 'q'
    elif price<400:
        return 'x'
    elif price<500:
        return 'y'
    else:
        return 'z'

In [36]:
#商品价格的onehot
goods_price_df = pd.read_excel('./good_feature_data/goods_feature_group.xlsx')
goods_df = pd.merge(goods_price_df,goods_feature_group_df,how='inner',on='goods_slug')
#去掉本来自带的单品价格编码，该用我们自己的
for i in range(77,92):
    del goods_df[i]
#交换列顺序，把price换到后面
columns_list = list(goods_df.columns)
del columns_list[1]
columns_list.append('price')
goods_df = goods_df[columns_list]
goods_df['price'] = goods_df['price'].apply(price_process)
goods_df = goods_df.join(pd.get_dummies(goods_df['price']))
del goods_df['price']

In [17]:
goods_class = pd.read_excel('./good_feature_data/goods_class.xlsx',sheet_name='Sheet1')

In [18]:
goods_class

,index,goods_slug,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_1,wo
0,0,s_ea_dcnx_240g,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,p_easyfun_qdszh_190726,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,s_shoushentaocan_1000k_190104,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,p_ea_3daypttp,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,s_easys_52_qingduanshi,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,s_cm25_yyb_384g,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,s_easyfun_zc_191028,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,s_easyfun_wc_21can,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,s_easyfun_wc_21wancan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,s_easyfun_qmmb_600g,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
goods_df.columns

Index(['goods_slug',            1,            2,            4,            5,
                  6,            7,            8,            9,           10,
                 11,           12,           13,           14,           15,
                 16,           17,           18,           19,           20,
                 21,           22,           23,           24,           25,
                 26,           27,           28,           29,           30,
                 31,           32,           33,           34,           35,
                 36,           37,           38,           39,           40,
                 41,           42,           43,           44,           45,
                 46,           47,           48,           49,           50,
                 51,           52,           53,           54,           55,
                 56,           57,           58,           59,           60,
                 61,           62,           63,           64,           65,